## Quantitative trading in China A stock market with FinRL

In [2]:
# !git clone https://github.com/AI4Finance-Foundation/FinRL-Meta
%cd FinRL-Meta/

/Users/zhengshuang/Documents/ProgramingGuide/SourceCode/GithubRepo/everyfine/FinRLLearn/FinRL-Tutorials-Learn/3-Practical/FinRL-Meta


/Users/zhengshuang/Documents/ProgramingGuide/SourceCode/GithubRepo/everyfine/FinRLLearn/FinRL-Tutorials-Learn/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


### Import modules

In [3]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
from IPython import display
display.set_matplotlib_formats("svg")

from meta import config
from meta.data_processors.tushare import Tushare, ReturnPlotter
from meta.env_stock_trading.env_stocktrading_China_A_shares import StockTradingEnv
from agents.stablebaselines3_models import DRLAgent
pd.options.display.max_columns = None
    
print("ALL Modules have been imported!")

ALL Modules have been imported!


### Create folders

In [4]:
import os
if not os.path.exists("./datasets" ):
    os.makedirs("./datasets" )
if not os.path.exists("./trained_models"):
    os.makedirs("./trained_models" )
if not os.path.exists("./tensorboard_log"):
    os.makedirs("./tensorboard_log" )
if not os.path.exists("./results" ):
    os.makedirs("./results" )

### Download data, cleaning and feature engineering

In [5]:
ticket_list=['600000.SH', '600009.SH', '600016.SH', '600028.SH', '600030.SH',
       '600031.SH', '600036.SH', '600050.SH', '600104.SH', '600196.SH',
       '600276.SH', '600309.SH', '600519.SH', '600547.SH', '600570.SH']

train_start_date='2015-01-01'
train_stop_date='2019-08-01'
val_start_date='2019-08-01'
val_stop_date='2021-01-03'

token='27080ec403c0218f96f388bca1b1d85329d563c91a43672239619ef5'

In [6]:
# download and clean
ts_processor = Tushare(data_source="tushare", 
                                   start_date=train_start_date,
                                   end_date=val_stop_date,
                                   time_interval="1d",
                                   token=token)
ts_processor.download_data(ticker_list=ticket_list)

  0%|          | 0/15 [00:00<?, ?it/s]

'close'
'close'


100%|██████████| 15/15 [00:18<00:00,  1.20s/it]

Download complete! Dataset saved to ./data/dataset.csv. 
Shape of DataFrame: (21574, 8)


In [7]:
ts_processor.clean_data()
ts_processor.fillna()
ts_processor.dataframe

Shape of DataFrame:  (21930, 8)


,tic,time,open,high,low,close,adjusted_close,volume
0,600000.SH,2015-01-05,15.88,16.25,15.56,16.07,16.07,5135687.09
1,600009.SH,2015-01-05,19.82,20.91,19.82,20.53,20.53,371485.54
2,600016.SH,2015-01-05,10.87,10.96,10.50,10.78,10.78,9138873.70
3,600028.SH,2015-01-05,6.59,7.14,6.45,7.14,7.14,11864996.45
4,600030.SH,2015-01-05,33.90,35.25,33.01,34.66,34.66,6986272.15
...,...,...,...,...,...,...,...,...
21925,600276.SH,2020-12-31,112.55,112.66,110.01,111.46,111.46,281568.85
21926,600309.SH,2020-12-31,89.88,91.48,88.70,91.04,91.04,213631.34
21927,600519.SH,2020-12-31,1941.00,1998.98,1939.00,1998.00,1998.00,38860.07
21928,600547.SH,2020-12-31,23.60,23.75,23.48,23.62,23.62,213786.94


In [8]:
# add_technical_indicator
ts_processor.add_technical_indicator(config.INDICATORS)
ts_processor.fillna()
ts_processor.dataframe

tech_indicator_list:  ['macd', 'boll_ub', 'boll_lb', 'rsi_30', 'cci_30', 'dx_30', 'close_30_sma', 'close_60_sma']
indicator:  macd
indicator:  boll_ub
indicator:  boll_lb
indicator:  rsi_30
indicator:  cci_30
indicator:  dx_30
indicator:  close_30_sma
indicator:  close_60_sma
Succesfully add technical indicators
Shape of DataFrame:  (21885, 17)


,tic,time,index,open,high,low,close,adjusted_close,volume,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma
0,600000.SH,2015-01-08,45,15.87,15.88,15.20,15.25,15.25,3306271.72,-0.032571,16.617911,15.012089,6.058641,-125.593009,23.014040,15.815000,15.815000
1,600009.SH,2015-01-08,46,20.18,20.18,19.73,20.00,20.00,198117.45,-0.016008,20.663897,19.736103,12.828915,-90.842491,100.000000,20.200000,20.200000
2,600016.SH,2015-01-08,47,10.61,10.66,10.09,10.20,10.20,4851684.17,-0.018247,10.957604,9.997396,11.862558,-99.887006,100.000000,10.477500,10.477500
3,600028.SH,2015-01-08,48,7.09,7.41,6.83,6.85,6.85,8190902.35,-0.008227,7.342000,6.743000,27.409248,36.578171,64.934862,7.042500,7.042500
4,600030.SH,2015-01-08,49,36.40,36.70,34.68,35.25,35.25,6376268.69,0.032910,36.576444,33.808556,61.517448,47.947020,100.000000,35.192500,35.192500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21880,600276.SH,2020-12-31,21925,112.55,112.66,110.01,111.46,111.46,281568.85,5.999563,120.049291,81.980709,65.497668,108.629006,49.657169,96.356667,92.992167
21881,600309.SH,2020-12-31,21926,89.88,91.48,88.70,91.04,91.04,213631.34,1.723646,92.780907,79.425093,58.501768,115.410537,38.129756,85.693000,84.104667
21882,600519.SH,2020-12-31,21927,1941.00,1998.98,1939.00,1998.00,1998.00,38860.07,42.535547,1942.989267,1761.052733,66.422960,220.691463,52.684229,1815.039333,1761.921500
21883,600547.SH,2020-12-31,21928,23.60,23.75,23.48,23.62,23.62,213786.94,-0.252554,24.495355,23.032645,42.060734,-55.475296,0.003164,23.883667,24.534000


### Split traning dataset

In [9]:
train = ts_processor.data_split(ts_processor.dataframe,train_start_date,train_stop_date)
len(train.tic.unique())

15

In [10]:
train.tic.unique()

array(['600000.SH', '600009.SH', '600016.SH', '600028.SH', '600030.SH',
       '600031.SH', '600036.SH', '600050.SH', '600104.SH', '600196.SH',
       '600276.SH', '600309.SH', '600519.SH', '600547.SH', '600570.SH'],
      dtype=object)

In [11]:
train.shape

(16695, 17)

In [15]:
train = train.rename(columns={'time': 'date'})
train.head()

,tic,date,index,open,high,low,close,adjusted_close,volume,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma
0,600000.SH,2015-01-08,45,15.87,15.88,15.20,15.25,15.25,3306271.72,-0.032571,16.617911,15.012089,6.058641,-125.593009,23.014040,15.8150,15.8150
0,600009.SH,2015-01-08,46,20.18,20.18,19.73,20.00,20.00,198117.45,-0.016008,20.663897,19.736103,12.828915,-90.842491,100.000000,20.2000,20.2000
0,600016.SH,2015-01-08,47,10.61,10.66,10.09,10.20,10.20,4851684.17,-0.018247,10.957604,9.997396,11.862558,-99.887006,100.000000,10.4775,10.4775
0,600028.SH,2015-01-08,48,7.09,7.41,6.83,6.85,6.85,8190902.35,-0.008227,7.342000,6.743000,27.409248,36.578171,64.934862,7.0425,7.0425
0,600030.SH,2015-01-08,49,36.40,36.70,34.68,35.25,35.25,6376268.69,0.032910,36.576444,33.808556,61.517448,47.947020,100.000000,35.1925,35.1925


In [16]:
stock_dimension = len(train.tic.unique())
state_space = stock_dimension*(len(config.INDICATORS)+2)+1
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 15, State Space: 151


### Train

In [17]:
env_kwargs = {
    "stock_dim": stock_dimension,
    "hmax": 1000, 
    "initial_amount": 1000000, 
    "buy_cost_pct":6.87e-5,
    "sell_cost_pct":1.0687e-3,
    "reward_scaling": 1e-4,
    "state_space": state_space, 
    "action_space": stock_dimension,
    "tech_indicator_list": config.INDICATORS, 
    "print_verbosity": 1,
    "initial_buy":True,
    "hundred_each_trade":False
}

e_train_gym = StockTradingEnv(df = train, **env_kwargs)